In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import copy
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from ipywidgets import interact

# **0. ResNet 모델 개요**

- 일반적으로는 신경망의 깊이가 깊어질수록 딥러닝 성능이 좋아짐
  > "Deep Residual Learning for Image Recognition" 논문에 의하면 신경망은 깊이가 깊어질수록 성능이 좋아지다가 일정한 단계에 다다르면 오히려 성능이 나빠진다고 함
  - 깊어진 신경망을 효과적으로 학습하기 위한 방법으로 **레지듀얼(residual, 잔차)** 개념 도입
- Residual block을 이용해 기울기가 잘 전파될 수 있도록 일종의 숏컷을 만들어 줌
  > 기울기 소멸 문제 방지
- VGG19 구조를 뼈대로 하며, 거기에 합성곱 층들을 추가해서 깊게 만든 후 숏컷들을 추가하는 것

### **용어 정리**

#### **1. 블록(block)**  
- 계층의 묶음
- 여러 합성곱 층을 하나로 묶은 단위

#### **2. 레지듀얼 블록(residual block)**
- 기울기가 잘 전파될 수 있도록 일종의 숏컷(shortcut)을 만들어 주는 것
- 기울기 소멸 방지
- 여러 계층을 묶어 하나의 레지듀얼 블록 생성
    - 레지듀얼 블록을 여러 개 쌓은 것이 ResNet
    
#### **3. 병목 블록(bottleneck block)**
- 계층의 깊이가 깊어질 때 파라미터의 수가 무제한으로 커지는 것을 방지
    - ResNet50의 경우 깊이가 깊어졌음에도 파라미터의 수가 감소
- 3x3 합성곱층의 앞뒤로 1x1 합성곱층을 붙임
    - 1x1 합성곱층의 채널 수를 조절하면서 차원 확대/축소

#### **4. 아이덴티티 매핑(identity mapping, 숏컷(shortcut))**
- 입력 x가 어떤 함수를 통과하더라도 다시 x라는 형태로 출력되도록 하는 것
- 아이덴티티 블록: 입력 차원 == 출력 차원

#### **5. 다운샘플(down-sample)**
- 특성 맵 크기를 줄이기 위한 것
    - 풀링과 같은 역할
- 아이덴티티 매핑을 적용하기 위해 블록을 넘어갈 때 형태를 맞춰주는 역할


# **1. ResNet 모델 불러오기**

In [2]:
model = models.resnet50(pretrained = True) # 이미 학습된 모델 사용

c:\users\doroc\appdata\local\programs\python\python37\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\users\doroc\appdata\local\programs\python\python37\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from torchsummary import summary # 모델 구조 요약

In [4]:
summary(model,input_size = (3,224,224),batch_size = 1,device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 112, 112]           9,408
       BatchNorm2d-2          [1, 64, 112, 112]             128
              ReLU-3          [1, 64, 112, 112]               0
         MaxPool2d-4            [1, 64, 56, 56]               0
            Conv2d-5            [1, 64, 56, 56]           4,096
       BatchNorm2d-6            [1, 64, 56, 56]             128
              ReLU-7            [1, 64, 56, 56]               0
            Conv2d-8            [1, 64, 56, 56]          36,864
       BatchNorm2d-9            [1, 64, 56, 56]             128
             ReLU-10            [1, 64, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]          16,384
      BatchNorm2d-12           [1, 256, 56, 56]             512
           Conv2d-13           [1, 256, 56, 56]          16,384
      BatchNorm2d-14           [1, 256,

# **2. 데이터에 맞게 모델의 Head 부분 수정하기**

In [5]:
model.avgpool = nn.AdaptiveAvgPool2d(output_size = (1,1))
model.fc = nn.Linear(2048,7) # 7개의 감정으로 분류

In [6]:
### 모델 생성 함수
def build_resnet50_based_model(device_name = 'cpu'):
    device = torch.device(device_name)
    model = models.resnet50(pretrained = True) # 이미 학습된 resnet50 모델 불러오기
    # 일반 NN layer(FC layer)
    model.avgpool = nn.AdaptiveAvgPool2d(output_size = (1,1))
    model.fc = nn.Linear(2048,7) # 7개의 감정으로 분류
    
    return model.to(device)

In [7]:
model = build_resnet50_based_model(device_name = 'cpu') # 모델 객체 생성

In [8]:
summary(model,(3,224,224),batch_size = 1,device = 'cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 112, 112]           9,408
       BatchNorm2d-2          [1, 64, 112, 112]             128
              ReLU-3          [1, 64, 112, 112]               0
         MaxPool2d-4            [1, 64, 56, 56]               0
            Conv2d-5            [1, 64, 56, 56]           4,096
       BatchNorm2d-6            [1, 64, 56, 56]             128
              ReLU-7            [1, 64, 56, 56]               0
            Conv2d-8            [1, 64, 56, 56]          36,864
       BatchNorm2d-9            [1, 64, 56, 56]             128
             ReLU-10            [1, 64, 56, 56]               0
           Conv2d-11           [1, 256, 56, 56]          16,384
      BatchNorm2d-12           [1, 256, 56, 56]             512
           Conv2d-13           [1, 256, 56, 56]          16,384
      BatchNorm2d-14           [1, 256,

# **3. 손실함수 정의**

In [9]:
loss_func = nn.CrossEntropyLoss(reduction = 'mean')

# **4. 옵티마이져 정의**

In [10]:
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3,momentum = 0.9)